# Heroes of Pymoli
## Data Analysis
### Matthew Stewart - March 16, 2019
---

### Initial Setup

In [265]:
# Import Dependencies
import pandas as pd
import os

In [266]:
# Establish input file and set up initial dataframe
inputFile = os.path.join('Resources', 'purchase_data.csv')

purchase_df = pd.read_csv(inputFile)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [267]:
# Check for possible missing information
purchase_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [268]:
# Check datatypes
purchase_df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

---

### Player Count

* According to the following, there are 576 total players.

In [269]:
# Calculate total number of players
totalPlayers = len(purchase_df['SN'].unique())
totalPlayers_df = pd.DataFrame({'Number of Players': [totalPlayers]})
totalPlayers_df

,Number of Players
0,576


---

### Purchasing Analysis (Total)
* The following illustrates:
 * Number of Unique Items
 * Average Purchase Price
 * Total Number of Purchases
 * Total Revenue

In [270]:
# Calculate number of unique items, average purchase price, total number of purchases, and total revenue
numUniqueItems = len(purchase_df['Item ID'].unique())
averagePurchasePrice = purchase_df['Price'].mean()
numPurchases = purchase_df['Purchase ID'].count()
totalRevenue = purchase_df['Price'].sum()

# Assign above variables into dictionary
totalPurchase_dict = {
    'Number of Unique Items': [numUniqueItems],
    'Average Purchase Price': [averagePurchasePrice],
    'Number of Purchases': [numPurchases],
    'Total Revenue': [totalRevenue]
}

# Pass above dictionary into dataframe
totalPurchase_df = pd.DataFrame(totalPurchase_dict, columns = [
    'Number of Unique Items',
    'Average Purchase Price',
    'Number of Purchases',
    'Total Revenue'
])

# Format cleanup
totalPurchase_df['Average Purchase Price'] = totalPurchase_df['Average Purchase Price'].map('${:,.2f}'.format)
totalPurchase_df['Total Revenue'] = totalPurchase_df['Total Revenue'].map('${:,.2f}'.format)

totalPurchase_df.head()

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


---

### Gender Demographics
* The following illustrates the number and percentage of players, broken down by gender.

In [271]:
# Check list of unique gender values
list(purchase_df['Gender'].unique())

['Male', 'Other / Non-Disclosed', 'Female']

In [272]:
# Calculate percent and count of players by gender
malePlayers_df = purchase_df.loc[purchase_df['Gender'] == 'Male']
numMalePlayers = len(malePlayers_df['SN'].unique())

femalePlayers_df = purchase_df.loc[purchase_df['Gender'] == 'Female']
numFemalePlayers = len(femalePlayers_df['SN'].unique())

otherPlayers_df = purchase_df.loc[purchase_df['Gender'] == 'Other / Non-Disclosed']
numOtherPlayers = len(otherPlayers_df['SN'].unique())

# For pecent calculations, the denominator (Total Players) was already calculated above
percentMalePlayers = '{:.2%}'.format(numMalePlayers / totalPlayers)
percentFemalePlayers = '{:.2%}'.format(numFemalePlayers / totalPlayers)
percentOtherPlayers = '{:.2%}'.format(numOtherPlayers / totalPlayers)

# Summary dataframe
gender_dict = {
    'Gender': ['Male', 'Female', 'Other / Non-Disclosed'],
    'Total Count': [numMalePlayers, numFemalePlayers, numOtherPlayers],
    'Percentage of Players': [percentMalePlayers, percentFemalePlayers, percentOtherPlayers]
}

gender_df = pd.DataFrame(gender_dict, columns = ['Gender', 'Total Count', 'Percentage of Players'])
gender_df = gender_df.set_index('Gender')

gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


---

### Purchasing Analysis (Gender)
* The following groups by Gender and illustrates:
 * Purchase Count
 * Average Purchase Price
 * Total Purchase Value
 * Average Total Purchase per Person (still broken down by Gender)

In [273]:
# Calculate purchase count, average purchase price, total purchase value,
# and average purchase total per person, all broken down by gender

# Create groupby object (by Gender) and calculate count, mean, and sum aggregtaes
# Convert the aggregate results into their own respective dataframes (for future merging)
purchase_df_grouped = purchase_df.groupby('Gender')
purchaseCount_gender = pd.DataFrame(purchase_df_grouped['Purchase ID'].count())
averagePrice_gender = pd.DataFrame(purchase_df_grouped['Price'].mean())
totalPrice_gender = pd.DataFrame(purchase_df_grouped['Price'].sum())

# Merge individual dataframes into one single dataframe and clean up column names and formatting
finalGenderPurchase_df = purchaseCount_gender.merge(averagePrice_gender, on = 'Gender')\
                            .merge(totalPrice_gender, on = 'Gender')\
                            .merge(gender_df, on = 'Gender') # gender_df is dataframe from cell above

finalGenderPurchase_df.rename(columns = {
    'Purchase ID': 'Purchase Count',
    'Price_x': 'Average Purchase Price',
    'Price_y': 'Total Purchase Value'
}, inplace = True)

finalGenderPurchase_df['Avg Total Purchase per Person'] = finalGenderPurchase_df['Total Purchase Value'] / \
                                                            finalGenderPurchase_df['Total Count']

finalGenderPurchase_df['Average Purchase Price'] = finalGenderPurchase_df['Average Purchase Price']\
                                                    .map('${:,.2f}'.format)

finalGenderPurchase_df['Total Purchase Value'] = finalGenderPurchase_df['Total Purchase Value']\
                                                    .map('${:,.2f}'.format)

finalGenderPurchase_df['Avg Total Purchase per Person'] = finalGenderPurchase_df['Avg Total Purchase per Person']\
                                                    .map('${:,.2f}'.format)

# Display desired results
finalGenderPurchase_df[[
    'Purchase Count',
    'Average Purchase Price',
    'Total Purchase Value',
    'Avg Total Purchase per Person'
]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


---

### Age Demographics
* The following illustrates the breakdown of player counts, grouped into age bins.

In [274]:
# Add age bins to the original dataset and group by these new age bins

# Check age min and max
print(purchase_df['Age'].min())
print(purchase_df['Age'].max())

7
45


In [275]:
# Create age bins and labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
binLabels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add age groups to original dataset
purchase_df['Age Group'] = pd.cut(purchase_df['Age'], bins, labels = binLabels)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [276]:
# Filter by each Age Group
ageGroupOne = purchase_df.loc[purchase_df['Age Group'] == '<10']
ageGroupTwo = purchase_df.loc[purchase_df['Age Group'] == '10-14']
ageGroupThree = purchase_df.loc[purchase_df['Age Group'] == '15-19']
ageGroupFour = purchase_df.loc[purchase_df['Age Group'] == '20-24']
ageGroupFive = purchase_df.loc[purchase_df['Age Group'] == '25-29']
ageGroupSix = purchase_df.loc[purchase_df['Age Group'] == '30-34']
ageGroupSeven = purchase_df.loc[purchase_df['Age Group'] == '35-39']
ageGroupEight = purchase_df.loc[purchase_df['Age Group'] == '40+']

# Calculate unique players per filtered group
numPlayersOne = len(ageGroupOne['SN'].unique())
numPlayersTwo = len(ageGroupTwo['SN'].unique())
numPlayersThree = len(ageGroupThree['SN'].unique())
numPlayersFour = len(ageGroupFour['SN'].unique())
numPlayersFive = len(ageGroupFive['SN'].unique())
numPlayersSix = len(ageGroupSix['SN'].unique())
numPlayersSeven = len(ageGroupSeven['SN'].unique())
numPlayersEight = len(ageGroupEight['SN'].unique())

# For percentage calculations, the denominator (Total Players) has already been calculated above
percentPlayersOne = '{:.2%}'.format(numPlayersOne / totalPlayers)
percentPlayersTwo = '{:.2%}'.format(numPlayersTwo / totalPlayers)
percentPlayersThree = '{:.2%}'.format(numPlayersThree / totalPlayers)
percentPlayersFour = '{:.2%}'.format(numPlayersFour / totalPlayers)
percentPlayersFive = '{:.2%}'.format(numPlayersFive / totalPlayers)
percentPlayersSix = '{:.2%}'.format(numPlayersSix / totalPlayers)
percentPlayersSeven = '{:.2%}'.format(numPlayersSeven / totalPlayers)
percentPlayersEight = '{:.2%}'.format(numPlayersEight / totalPlayers)

# Final age demographics dataframe
ageGroup_dict = {
    'Age Group': ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'],
    'Total Count': [numPlayersOne, numPlayersTwo, numPlayersThree, numPlayersFour,
                    numPlayersFive, numPlayersSix, numPlayersSeven, numPlayersEight],
    'Percentage of Players': [percentPlayersOne, percentPlayersTwo, percentPlayersThree, percentPlayersFour,
                              percentPlayersFive, percentPlayersSix, percentPlayersSeven, percentPlayersEight]
}

ageGroup_df = pd.DataFrame(ageGroup_dict, columns = ['Age Group', 'Total Count', 'Percentage of Players'])
ageGroup_df.set_index('Age Group', inplace = True)

ageGroup_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


---

### Purchasing Analysis (Age)
* The following groups by Age Group and illustrates:
 * Purchase Count
 * Average Purchase Price
 * Total Purchase Value
 * Average Total Purchase per Person (still broken down by Age Group)

In [277]:
# Calculate purchase count, average purchase price, total purchase value,
# and average purchase total per person, all broken down by the age group bins created above

# Create groupby object (by Age Group) and calculate count, mean, and sum aggregtaes
# Convert the aggregate results into their own respective dataframes (for future merging)
purchase_df_ageGroup = purchase_df.groupby('Age Group')
purchaseCount_ageGroup = pd.DataFrame(purchase_df_ageGroup['Purchase ID'].count())
averagePrice_ageGroup = pd.DataFrame(purchase_df_ageGroup['Price'].mean())
totalPrice_ageGroup = pd.DataFrame(purchase_df_ageGroup['Price'].sum())

# Merge individual dataframes into one single dataframe and clean up column names and formatting
finalAgeGroup_df = purchaseCount_ageGroup.merge(averagePrice_ageGroup, on = 'Age Group')\
                            .merge(totalPrice_ageGroup, on = 'Age Group')\
                            .merge(ageGroup_df, on = 'Age Group') # ageGroup_df is dataframe from cell above

finalAgeGroup_df.rename(columns = {
    'Purchase ID': 'Purchase Count',
    'Price_x': 'Average Purchase Price',
    'Price_y': 'Total Purchase Value'
}, inplace = True)

finalAgeGroup_df['Avg Total Purchase per Person'] = finalAgeGroup_df['Total Purchase Value'] / \
                                                            finalAgeGroup_df['Total Count']

finalAgeGroup_df['Average Purchase Price'] = finalAgeGroup_df['Average Purchase Price']\
                                                    .map('${:,.2f}'.format)

finalAgeGroup_df['Total Purchase Value'] = finalAgeGroup_df['Total Purchase Value']\
                                                    .map('${:,.2f}'.format)

finalAgeGroup_df['Avg Total Purchase per Person'] = finalAgeGroup_df['Avg Total Purchase per Person']\
                                                    .map('${:,.2f}'.format)

# Display desired results
finalAgeGroup_df[[
    'Purchase Count',
    'Average Purchase Price',
    'Total Purchase Value',
    'Avg Total Purchase per Person'
]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


---

### Top Spenders
* The following illustrates the top 5 spenders by total purchase value, in addition to purchase count and average price.

In [278]:
# Create groupby object (by SN) and calculate count, mean, and sum aggregtaes
# Convert the aggregate results into their own respective dataframes (for future merging)
purchase_df_snGroup = purchase_df.groupby('SN')
purchaseCount_snGroup = pd.DataFrame(purchase_df_snGroup['Purchase ID'].count())
averagePrice_snGroup = pd.DataFrame(purchase_df_snGroup['Price'].mean())
totalPrice_snGroup = pd.DataFrame(purchase_df_snGroup['Price'].sum())

# Merge individual dataframes into one single dataframe and clean up column names and formatting
topSpender_df = purchaseCount_snGroup.merge(averagePrice_snGroup, on = 'SN')\
                            .merge(totalPrice_snGroup, on = 'SN')

topSpender_df.rename(columns = {
    'Purchase ID': 'Purchase Count',
    'Price_x': 'Average Purchase Price',
    'Price_y': 'Total Purchase Value'
}, inplace = True)

topSpender_df.sort_values(by = 'Total Purchase Value', ascending = False, inplace = True)

topSpender_df['Average Purchase Price'] = topSpender_df['Average Purchase Price']\
                                                    .map('${:,.2f}'.format)

topSpender_df['Total Purchase Value'] = topSpender_df['Total Purchase Value']\
                                                    .map('${:,.2f}'.format)

topSpender_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


---

### Most Popular Items
* The following illustrates the top 5 items by purchase count, in addition to item price and total purchase value.

In [279]:
# Create groupby object (by Item ID, Item Name, and Price) and calculate count aggregate
purchase_df_itemGroup = purchase_df.groupby(['Item ID', 'Item Name', 'Price'])

# Create temporary (interim) dataframe that will (1) calculate the count aggregate,
# (2) turn the "Price" index back into a column, and (3) re-order the columns
item_interimCount_df = pd.DataFrame(purchase_df_itemGroup['Purchase ID'].count())

item_interimCount_df.reset_index(inplace = True)
item_interimCount_df.set_index(['Item ID', 'Item Name'], inplace = True)
item_interimCount_df.rename(columns = {
    'Price': 'Item Price',
    'Purchase ID': 'Purchase Count'
}, inplace = True)
item_interimCount_df = item_interimCount_df[['Purchase Count', 'Item Price']]

# Create a second groupby object (by Item ID and Item Name)
purchase_df_itemGroup2 = purchase_df.groupby(['Item ID', 'Item Name'])

# Create second iterim dataframe that will calculate sum aggregate
item_interimSum_df = pd.DataFrame(purchase_df_itemGroup2['Price'].sum())
item_interimSum_df.rename(columns = {
    'Price': 'Total Purchase Value'
}, inplace = True)

# Merge the two interim dataframes
topItem_df = item_interimCount_df.merge(item_interimSum_df, on = ['Item ID', 'Item Name'])

# Sort and format
topItem_df.sort_values(by = 'Purchase Count', ascending = False, inplace = True)
topItem_df['Item Price'] = topItem_df['Item Price'].map('${:.2f}'.format)
topItem_df['Total Purchase Value'] = topItem_df['Total Purchase Value'].map('${:.2f}'.format)
topItem_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


---

### Most Profitable Items
* The following illustrates the top 5 items by total purchase value, in addition to item price and purchase count.

In [280]:
# Sort the "Most Popular Items" table above by Total Purchase Value instead of Purchase Count

# Delete the '$' and convert Total Purchase Value back to a float
mostProfitableItem_df = topItem_df.copy()
mostProfitableItem_df['Total Purchase Value'].replace('[\$,)]','', regex=True, inplace = True)
mostProfitableItem_df['Total Purchase Value'] = mostProfitableItem_df['Total Purchase Value'].astype('float')

# Sort by Total Purchase Value and add '$' back in
mostProfitableItem_df.sort_values(by = 'Total Purchase Value', ascending = False, inplace = True)
mostProfitableItem_df['Total Purchase Value'].map('${:.2f}'.format)
mostProfitableItem_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,50.76
82,Nirvana,9,$4.90,44.10
145,Fiery Glass Crusader,9,$4.58,41.22
92,Final Critic,8,$4.88,39.04
103,Singed Scalpel,8,$4.35,34.80
